In [1]:
import pandas as pd
import os
from sklearn.utils import shuffle
import glob
import itertools
import string
#import spacy
#nlp = spacy.load('en')

import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import datetime as dt

In [2]:
def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary


data_index = 0
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, context

####Helper fucntion to clean sentences####
def clean_list_of_words(list_of_words):   
    words = [word.lower() for word in list_of_words]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in words]
    return stripped

####Helper function to clean float to string####
def int_or_float(s):
    try:
        return str(s)
    except ValueError:
        return float(s)


In [3]:
fpath = str(os.getcwd()) + '/Data/regents'
all_files = glob.glob(fpath + "/*.tsv")
list_of_dfs = [pd.read_csv(file, sep='\t') for file in all_files]

    
#Create the header corpus
#Collect the header items into a list 
#flatten the list
#Remove header with Unnamed in it
header_list=[]
body_list = []
cleaned_header_corpus = []
cleaned_body_corpus = []

for df in list_of_dfs:
    
    #####################Create the header corpus######################
    header_list.append(df.columns.values)
    flattened_header_list = [item for sublist in header_list for item in sublist]
    flattened_header_list_clean = [item for item in flattened_header_list if not 'Unnamed' in item]
    #print(flattened_header_list_clean) 
   
    #Convert words into lower case
    words = [word.lower() for word in flattened_header_list_clean]
    #print(words[:100])
    #Strip the punctuation from the table
    table = str.maketrans('', '', string.punctuation)
    cleaned_header_corpus.append([w.translate(table) for w in words]) 
    #print(cleaned_header_corpus[:100])
    ################################################################## 
      
    
    #####################Create the body corpus######################
    #Convert the body items to list
    body_list = df.values.tolist()
    #Flatten the listoflist to a list
    flattened_body_list= [item for sublist in body_list for item in sublist] 
    #Convert floats in the list to int
    flattened_body_strlist = [int_or_float(item) for item in flattened_body_list]
    #Convert words to lowercase
    body_words_lowerlist = [word.lower() for word in flattened_body_strlist]   
    cleaned_body_corpus.append(clean_list_of_words(body_words_lowerlist))
    
    #print(cleaned_body_corpus[:100])
    ##################################################################
    
#print(cleaned_body_corpus[:2])
#print(cleaned_header_corpus)

########Flattened Header and Body Corpus##############################
merged_body_corpus = list(itertools.chain.from_iterable(cleaned_body_corpus))
merged_header_corpus = list(itertools.chain.from_iterable(cleaned_header_corpus))


vocabulary_size = 50000
data, count, dictionary, reverse_dictionary = build_dataset(
    merged_body_corpus, vocabulary_size) # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 0], ('is', 519), ('for health', 316), ('is located in the', 267), ('is a', 224)]
Sample data [362, 363, 1, 80, 41, 364, 98, 59, 41, 365] ['an', 'facial scar', 'is', 'acquired', 'nan', 'blue eyes', 'are', 'inherited', 'nan', 'long hair']


In [4]:
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2  

In [5]:
batch, labels = generate_batch(data,batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
        reverse_dictionary[labels[i, 0]])

363 facial scar -> 1 is
363 facial scar -> 362 an
1 is -> 80 acquired
1 is -> 363 facial scar
80 acquired -> 1 is
80 acquired -> 41 nan
41 nan -> 364 blue eyes
41 nan -> 80 acquired


In [6]:
graph = tf.Graph()
with graph.as_default():
    
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)


    # Construct the variables for the softmax
    weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                              stddev=1.0 / math.sqrt(embedding_size)))
    
    biases = tf.Variable(tf.zeros([vocabulary_size]))
    hidden_out = tf.matmul(embed, tf.transpose(weights)) + biases
    
    
    # convert train_context to a one-hot format
    train_one_hot = tf.one_hot(train_context, vocabulary_size)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hidden_out, 
        labels=train_one_hot))
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(cross_entropy)
    
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [13]:
def run(graph, num_steps):
    with tf.Session(graph=graph) as session:
      # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(data,
                batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, cross_entropy], feed_dict=feed_dict)
        average_loss += loss_val
        
        print(average_loss)
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0
            
            
    # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
        final_embeddings = normalized_embeddings.eval()
        

In [14]:
num_steps = 100
softmax_start_time = dt.datetime.now()
run(graph, num_steps=num_steps)
softmax_end_time = dt.datetime.now()
print("Softmax method took {} minutes to run 100 iterations".format((softmax_end_time-softmax_start_time).total_seconds()))

Initialized
11.09147834777832
Softmax method took 0.517762 minutes to run 100 iterations
